### Forecasting Mini-Course Sales
特徴量生成 + 基本的に2020年は削除 (https://www.kaggle.com/code/gauthamupadhyaya/mini-course-sales)  
~~2020年3-6月は何かで代用したい~~ (https://www.kaggle.com/code/yeoyunsianggeremie/s3e19-catboost-smoothing-post-processing)
 
- 学習は2021年を予測
- 国、店舗、製品を分けて学習してみる
- 前年の売上げを特徴量として組み込まない
- (train, valid, test ラベルを付けて、全部くっつけて特徴量を生成してみる)
- ~~スプライン形状、~~ 休日情報を付与

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import SplineTransformer
from holidays import CountryHoliday
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [14]:
CountryHoliday(country="JPN", years=2022)

{datetime.date(2022, 1, 1): '元日', datetime.date(2022, 1, 10): '成人の日', datetime.date(2022, 2, 11): '建国記念の日', datetime.date(2022, 2, 23): '天皇誕生日', datetime.date(2022, 3, 21): '春分の日', datetime.date(2022, 4, 29): '昭和の日', datetime.date(2022, 5, 3): '憲法記念日', datetime.date(2022, 5, 4): 'みどりの日', datetime.date(2022, 5, 5): 'こどもの日', datetime.date(2022, 7, 18): '海の日', datetime.date(2022, 8, 11): '山の日', datetime.date(2022, 9, 19): '敬老の日', datetime.date(2022, 9, 23): '秋分の日', datetime.date(2022, 10, 10): 'スポーツの日', datetime.date(2022, 11, 3): '文化の日', datetime.date(2022, 11, 23): '勤労感謝の日'}

In [2]:
import make_graph    # 自前
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [32]:
df_train = pd.read_csv("./data/train.csv", parse_dates=["date"])
df_test = pd.read_csv("./data/test.csv", parse_dates=["date"])
df_subm = pd.read_csv("./data/sample_submission.csv")

In [33]:
df_test["num_sold"] = np.nan
df_all = pd.concat([df_train, df_test], ignore_index=True)

df_all['year'] = df_all['date'].dt.year
df_all['month'] = df_all['date'].dt.month
df_all['day'] = df_all['date'].dt.day

# df_all["type"] = "train"
# df_all["type"] = np.where(df_all["year"]==2021, "valid", df_all["type"])
# df_all["type"] = np.where(df_all["year"]==2022, "test", df_all["type"])

In [34]:
print(df_all.shape)
df_all.head(5)

(164325, 9)


,id,date,country,store,product,num_sold,year,month,day
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63.0,2017,1,1
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66.0,2017,1,1
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9.0,2017,1,1
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59.0,2017,1,1
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49.0,2017,1,1


In [6]:
# # 2020年3-6月を置換 (2020年も含めて平均してみる)
# df_buf = df_all[(df_all["year"]<2021) & (df_all["month"].isin([3,4,5,6]))]
# df_buf = df_buf.groupby(by=["month", "day", "country", "store", "product"])["num_sold"].mean().reset_index()
# df_buf["year"] = 2020
# df_buf = pd.merge(df_all[["id", "year", "month", "day", "country", "store", "product"]], df_buf).sort_values("id")
# df_all.loc[((df_all["year"]==2020) & (df_all["month"].isin([3,4,5,6]))), "num_sold"] = df_buf["num_sold"].values    # df_allはソート済なので、そのまま置換
# print(df_all.shape)

(164325, 10)


In [76]:
# make_graph.trends(df_all, "num_sold", 1200, 300, "graph1.html", "コロナ置換")

In [35]:
CSP = {col: list(df_all[col].unique()) for col in ["country", "store", "product"]}
CSP

{'country': ['Argentina', 'Canada', 'Estonia', 'Japan', 'Spain'],
 'store': ['Kaggle Learn', 'Kaggle Store', 'Kagglazon'],
 'product': ['Using LLMs to Improve Your Coding',
  'Using LLMs to Train More LLMs',
  'Using LLMs to Win Friends and Influence People',
  'Using LLMs to Win More Kaggle Competitions',
  'Using LLMs to Write Better']}

In [36]:
def select_csp(df, idx):
    """インデックス指定で各名称取得、DataFrameフィルタ"""
    if len(idx) != 3:
        return (), pd.DataFrame()
    df_ret = df.copy()
    for i, col in zip(idx, ["country", "store", "product"]):
        if i < 0:
            continue
        idx_buf = i
        if i >= len(CSP[col]):
            print(f"[error] 指定インデックスが範囲外です。 ({col}: {CSP[col]})")
            idx_buf = 0
        df_ret = df_ret[df_ret[col] == CSP[col][idx_buf]]

    sel = {col: list(df_ret[col].unique()) for col in ["country", "store", "product"]}
    return sel, df_ret

In [8]:
csp, df_buf = select_csp(df_all, [-1, 0, 0])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]}"
p = make_graph.make_trend(df_buf, "num_sold", "country", 900, 300, ttl)
show(p)

In [37]:
def make_dummies(df_temp, cat_cols):
    """国、店舗、製品情報をダミー変数化"""
    df_dummy = pd.get_dummies(df_temp[cat_cols]).astype(np.uint8)
    df_temp = pd.merge(df_temp, df_dummy, left_index=True, right_index=True)
    print("dummies:", df_temp.shape)
    return df_temp

In [38]:
def datetime_features(df_temp):
    """時間情報の特徴量生成"""
    df_temp['dayofyear'] = df_temp['date'].dt.day_of_year
    df_temp['dayofweek'] = df_temp['date'].dt.dayofweek
    df_temp['quarter'] = df_temp['date'].dt.quarter
    # df_temp['weekofyear'] = df_temp['date'].dt.weekofyear  # 見つからない
    df_temp['weekofyear'] = df_temp['date'].apply(lambda x: x.isocalendar()[1])
    df_temp['is_weekend'] = (df_temp['dayofweek'] >= 5).astype(np.int8)
    df_temp['is_month_start'] = df_temp['date'].dt.is_month_start.astype(np.int8)
    df_temp['is_month_end'] = df_temp['date'].dt.is_month_end.astype(np.int8)
    # df_temp['monday'] = df_temp['date'].dt.weekday.eq(0).astype(np.uint8)
    # df_temp['tuesday'] = df_temp['date'].dt.weekday.eq(1).astype(np.uint8)
    # df_temp['wednesday'] = df_temp['date'].dt.weekday.eq(2).astype(np.uint8)
    # df_temp['thursday'] = df_temp['date'].dt.weekday.eq(3).astype(np.uint8)
    # df_temp['friday'] = df_temp['date'].dt.weekday.eq(4).astype(np.uint8)
    # df_temp['saturday'] = df_temp['date'].dt.weekday.eq(5).astype(np.uint8)
    # df_temp['sunday'] = df_temp['date'].dt.weekday.eq(6).astype(np.uint8)
    df_temp['dayname'] = df_temp['date'].dt.strftime("%A")    # 曜日。後でダミー変数化
    df_temp['is_quarter_end'] = df_temp['date'].dt.is_quarter_end.astype(np.uint8)
    df_temp['is_quarter_start'] = df_temp['date'].dt.is_quarter_start.astype(np.uint8)
    df_temp['is_year_end'] = df_temp['date'].dt.is_year_end.astype(np.uint8)
    df_temp['is_year_start'] = df_temp['date'].dt.is_year_start.astype(np.uint8)    
    print("datetime:", df_temp.shape)
    return df_temp

In [22]:
def GetAndAddHolidays(df):
    # Get country-specific holidays
    country_list = ['Argentina', 'Canada', 'Estonia', 'Japan', 'Spain']
    years = np.arange(df['year'].min(), df['year'].max() + 1)
    holidays = pd.DataFrame(columns=['date', 'holiday', 'country'])

    for country in country_list:
        for h in CountryHoliday(country, years=years).items():
            i=len(holidays)
            holidays.loc[i,'date']=pd.to_datetime(h[0])
            holidays.loc[i,'holiday']=h[1]
            holidays.loc[i,'country']=country

    # Add common holidays from Dec 24 to Dec 31 for each country
    common_holidays = pd.DataFrame({
    'date': pd.to_datetime(['2017-04-16','2017-12-24', '2017-12-25', '2017-12-26', '2017-12-27', '2017-12-28', '2017-12-29', '2017-12-30', '2017-12-31',
                            '2018-04-01','2018-12-24', '2018-12-25', '2018-12-26', '2018-12-27', '2018-12-28', '2018-12-29', '2018-12-30', '2018-12-31',
                            '2019-04-21','2019-12-24', '2019-12-25', '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29', '2019-12-30', '2019-12-31',
                            '2020-04-12','2020-12-24', '2020-12-25', '2020-12-26', '2020-12-27', '2020-12-28', '2020-12-29', '2020-12-30', '2020-12-31',
                            '2021-04-04','2021-12-24', '2021-12-25', '2021-12-26', '2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31',
                            '2022-04-17','2022-12-24', '2022-12-25', '2022-12-26', '2022-12-27', '2022-12-28', '2022-12-29', '2022-12-30', '2022-12-31']),
    'holiday': 'Holiday Season'
    })

    country_common_holidays = pd.DataFrame()
    for country in country_list:
        temp_common_holidays = common_holidays.copy()
        temp_common_holidays['country'] = country
        country_common_holidays = pd.concat([country_common_holidays, temp_common_holidays], ignore_index=True)

    holidays = pd.concat([holidays, country_common_holidays], ignore_index=True)
    holidays['date'] = pd.to_datetime(holidays['date'])
    holidays['isHoliday']=1
    
    
    # Merge holidays with the original DataFrame based on date and country
    df = pd.merge(df, holidays, how='left', on=['date', 'country'])
    df['holiday'] = df['holiday'].fillna('Not Holiday')
    df['isHoliday'] = df['isHoliday'].fillna(0)
    
    # Add Weekend columns
    #df['Friday'] = (df['dayofweek'] == 4).astype(int)
    #df['Saturday'] = (df['dayofweek'] == 5).astype(int)
    #df['Sunday'] = (df['dayofweek'] == 6).astype(int)
    
    #Quarter, weekly season
    #df['Q1'] = ((df['month'] >= 1) & (df['month'] <= 3)).astype(int)
    #df['Q2'] = ((df['month'] >= 4) & (df['month'] <= 6)).astype(int)
    #df['Q3'] = ((df['month'] >= 7) & (df['month'] <= 9)).astype(int)
    #df['Q4'] = ((df['month'] >= 10) & (df['month'] <= 12)).astype(int)

    # Remove duplicates from df
    df = df.drop_duplicates(subset=['date', 'country','product','store'], keep='first').reset_index().drop(columns = ["index"])

    print("holidays:", df.shape)
    return df, holidays

#full, Holidays = GetAndAddHolidays(full)

In [93]:
df_buf, holidays = GetAndAddHolidays(df_all)
df_buf.sample(5)

holidays: (164325, 23)


,id,date,country,store,product,num_sold,year,month,day,dayofyear,...,is_weekend,is_month_start,is_month_end,dayname,is_quarter_end,is_quarter_start,is_year_end,is_year_start,holiday,isHoliday
36302,36302,2018-04-30,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,5.0,2018,4,30,120,...,0,0,1,Monday,0,0,0,0,Feriado con fines turísticos,1.0
143343,143343,2022-03-27,Canada,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,NaN,2022,3,27,86,...,1,0,0,Sunday,0,0,0,0,Not Holiday,0.0
151844,151844,2022-07-18,Estonia,Kagglazon,Using LLMs to Write Better,NaN,2022,7,18,199,...,0,0,0,Monday,0,0,0,0,Not Holiday,0.0
110618,110618,2021-01-14,Spain,Kaggle Store,Using LLMs to Win More Kaggle Competitions,109.0,2021,1,14,14,...,0,0,0,Thursday,0,0,0,0,Not Holiday,0.0
982,982,2017-01-14,Argentina,Kaggle Store,Using LLMs to Win Friends and Influence People,9.0,2017,1,14,14,...,1,0,0,Saturday,0,0,0,0,Not Holiday,0.0


In [90]:
holidays

,date,holiday,country,isHoliday
0,2017-01-01,Año Nuevo,Argentina,1
1,2017-02-27,Día de Carnaval,Argentina,1
2,2017-02-28,Día de Carnaval,Argentina,1
3,2017-03-24,Día Nacional de la Memoria por la Verdad y la ...,Argentina,1
4,2017-04-02,Día del Veterano y de los Caidos en la Guerra ...,Argentina,1
...,...,...,...,...
690,2022-12-27,Holiday Season,Spain,1
691,2022-12-28,Holiday Season,Spain,1
692,2022-12-29,Holiday Season,Spain,1
693,2022-12-30,Holiday Season,Spain,1


In [39]:
def seasonality_features(df_temp):
    df_temp['month_sin'] = np.sin(2*np.pi*df_temp.month/12)
    df_temp['month_cos'] = np.cos(2*np.pi*df_temp.month/12)
    df_temp['day_sin'] = np.sin(2*np.pi*df_temp.day/24)    # 24 -> 30 -> 24
    df_temp['day_cos'] = np.cos(2*np.pi*df_temp.day/24)
    print("seasonality:", df_temp.shape)
    return df_temp

In [40]:
def make_features(df_temp):
    """全特徴量を追加"""
    df_temp = datetime_features(df_temp)
    df_temp, TrainHolidays = GetAndAddHolidays(df_all)
    df_temp = make_dummies(df_temp, ["country", "store", "product", "dayname", "holiday"])
    df_temp = seasonality_features(df_temp)
    return df_temp

# 特徴量生成
df_feat = make_features(df_all)
df_feat.head(3)

datetime: (164325, 21)
holidays: (164325, 23)
dummies: (164325, 129)
seasonality: (164325, 133)
(164325, 133)


,id,date,country,store,product,num_sold,year,month,day,dayofyear,...,holiday_敬老の日,holiday_文化の日,holiday_春分の日,holiday_昭和の日,holiday_海の日,holiday_秋分の日,month_sin,month_cos,day_sin,day_cos
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63.0,2017,1,1,1,...,0,0,0,0,0,0,0.5,0.866025,0.258819,0.965926
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66.0,2017,1,1,1,...,0,0,0,0,0,0,0.5,0.866025,0.258819,0.965926
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9.0,2017,1,1,1,...,0,0,0,0,0,0,0.5,0.866025,0.258819,0.965926


In [105]:
[col for col in df_feat.columns if "holiday_N" in col]

['holiday_Navidad',
 "holiday_New Year's Day",
 "holiday_New Year's Day (Observed)",
 'holiday_Not Holiday']

In [34]:
csp, df_buf = select_csp(df_feat, [1, 0, -1])
print(csp)

{'country': ['Canada'], 'store': ['Kaggle Learn'], 'product': ['Using LLMs to Improve Your Coding', 'Using LLMs to Train More LLMs', 'Using LLMs to Win Friends and Influence People', 'Using LLMs to Win More Kaggle Competitions', 'Using LLMs to Write Better']}


In [35]:
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]}"
p = make_graph.make_trend(df_buf, "num_sold", "product", 900, 300, ttl)
show(p)

各モデルの性能確認

In [28]:
import time
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, LinearRegression, ElasticNet
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedKFold, RepeatedStratifiedKFold, ShuffleSplit, cross_val_score
from sklearn.metrics import explained_variance_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.inspection import PartialDependenceDisplay
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [29]:
def get_smape(actual, forecast):
    """自前で評価値(SMAPE:https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error)産出"""
    smape = 100/len(actual) * np.sum(2 * np.abs(forecast - actual) / (np.abs(forecast) + np.abs(actual)))
    # smape = np.sum(np.abs(forecast - actual)) / np.sum(forecast + actual)
    return smape

In [45]:
# 学習とテスト用分離
remove_cols = ["id", "date", "country", "store", "product", "num_sold", "dayname", "holiday"]
train = df_feat[df_feat["year"]<2020].copy()
valid = df_feat[df_feat["year"]==2021].copy()

X_train = train.drop(columns=remove_cols)
y_train = train["num_sold"].values
X_valid = valid.drop(columns=remove_cols)
y_valid = valid["num_sold"].values
X_test = df_feat[df_feat["year"]==2022].drop(columns=remove_cols)
print(X_train.shape, len(y_train), X_valid.shape, len(y_valid), X_test.shape)

(82125, 125) 82125 (27375, 125) 27375 (27375, 125)


In [46]:
X_train.head(3)

,year,month,day,dayofyear,dayofweek,quarter,weekofyear,is_weekend,is_month_start,is_month_end,...,holiday_敬老の日,holiday_文化の日,holiday_春分の日,holiday_昭和の日,holiday_海の日,holiday_秋分の日,month_sin,month_cos,day_sin,day_cos
0,2017,1,1,1,6,1,52,1,1,0,...,0,0,0,0,0,0,0.5,0.866025,0.258819,0.965926
1,2017,1,1,1,6,1,52,1,1,0,...,0,0,0,0,0,0,0.5,0.866025,0.258819,0.965926
2,2017,1,1,1,6,1,52,1,1,0,...,0,0,0,0,0,0,0.5,0.866025,0.258819,0.965926


In [43]:
regressors = [LinearRegression(), 
              Lasso(), 
              DecisionTreeRegressor(), 
              Ridge(), 
              ElasticNet(), 
              RandomForestRegressor(), 
              XGBRegressor(), 
              CatBoostRegressor(), 
              LGBMRegressor(), 
              GradientBoostingRegressor(), 
              HistGradientBoostingRegressor()]

In [47]:
scores = []
for model in regressors:
    start = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start
    # start = time.time()
    y_pred = model.predict(X_valid)
    y_pred = ((y_pred * 2 + 1) // 2).astype(int)    # 四捨五入int
    # predict_time = time.time()-start
    name = str(model)[:-2]
    name = "XGBRegressor" if "XGBRegressor" in name else name
    name = "CatBoostRegressor" if "CatBoostRegressor" in name else name
    print(f"{name}, {train_time:.3f}s")
    scores.append([name, train_time, 
                   explained_variance_score(y_valid, y_pred),
                   mean_squared_error(y_valid, y_pred, squared=False), 
                   mean_absolute_error(y_valid, y_pred),
                   r2_score(y_valid, y_pred),
                   get_smape(y_valid, y_pred)])
df_score = pd.DataFrame(scores, columns=["model", "train_time", "explained_variance", "mae", "rmse", "r2", "smape"])

LinearRegression, 0.514s
Lasso, 1.054s
DecisionTreeRegressor, 0.772s
Ridge, 0.055s
ElasticNet, 0.922s
RandomForestRegressor, 48.118s
XGBRegressor, 4.194s
Learning rate set to 0.082161
0:	learn: 169.0945442	total: 66.6ms	remaining: 1m 6s
1:	learn: 156.3678195	total: 82.6ms	remaining: 41.2s
2:	learn: 144.6896662	total: 97.5ms	remaining: 32.4s
3:	learn: 134.1189866	total: 114ms	remaining: 28.4s
4:	learn: 124.4355586	total: 131ms	remaining: 26.1s
5:	learn: 115.7013394	total: 147ms	remaining: 24.3s
6:	learn: 107.7493396	total: 158ms	remaining: 22.4s
7:	learn: 100.4925176	total: 165ms	remaining: 20.5s
8:	learn: 93.9439421	total: 172ms	remaining: 19s
9:	learn: 87.9693269	total: 178ms	remaining: 17.6s
10:	learn: 82.6006976	total: 183ms	remaining: 16.4s
11:	learn: 77.7280951	total: 187ms	remaining: 15.4s
12:	learn: 73.2704969	total: 192ms	remaining: 14.6s
13:	learn: 69.3012482	total: 196ms	remaining: 13.8s
14:	learn: 65.7355198	total: 200ms	remaining: 13.1s
15:	learn: 62.5046717	total: 204ms	re

In [48]:
df_score.sort_values("smape", ascending=True)

,model,train_time,explained_variance,mae,rmse,r2,smape
5,RandomForestRegressor,48.118428,9.776598e-01,3.243491e+01,1.742287e+01,9.733098e-01,10.089537
7,CatBoostRegressor,3.361039,9.780170e-01,3.222838e+01,1.740658e+01,9.736486e-01,10.760235
2,DecisionTreeRegressor,0.772336,9.717851e-01,3.581888e+01,1.917626e+01,9.674500e-01,11.161771
10,HistGradientBoostingRegressor,1.541940,9.769840e-01,3.278399e+01,1.757816e+01,9.727322e-01,11.773279
6,XGBRegressor,4.194001,9.772529e-01,3.271067e+01,1.764676e+01,9.728540e-01,12.025172
8,LGBMRegressor,0.304723,9.771024e-01,3.268814e+01,1.755737e+01,9.728914e-01,12.613229
9,GradientBoostingRegressor,10.324080,9.593680e-01,4.169548e+01,2.402893e+01,9.558933e-01,25.447717
1,Lasso,1.053968,7.457170e-01,1.007913e+02,6.693271e+01,7.422653e-01,55.787023
3,Ridge,0.055211,7.512501e-01,9.930335e+01,6.832658e+01,7.498189e-01,57.611505
0,LinearRegression,0.513940,-1.985109e+11,8.860207e+07,5.080340e+06,-1.991656e+11,58.062889


In [49]:
def disp_scores(Y_test, test_pred):
    print("Test Scores")
    print("\tExplained variance:", explained_variance_score(Y_test, test_pred))
    print("\tMean absolute error:", mean_absolute_error(Y_test, test_pred))
    print("\tMean squared error:", mean_squared_error(Y_test, test_pred))
    print("\tR2 score:", r2_score(Y_test, test_pred))
    print("\tSMAPE score:", get_smape(Y_test, test_pred))

In [50]:
def get_prediction(train, valid, test, model):
    """各予測値を返す"""
    # # スケーリングして学習させたので、逆変換する
    # return [minmax.inverse_transform(model.predict(x).reshape(-1,1)) for x in [train, valid, test]]
    # 小さい値で差分が出るのを抑えるため、四捨五入してみる
    ret_arr = []
    for x in [train, valid, test]:
        prediction = model.predict(x)
        ret_arr.append(((prediction * 2 + 1) // 2).astype(int))
    return ret_arr

def learning(model, X_train, Y_train, X_valid, Y_valid, X):
    """学習、予測値出力"""
    model.fit(X_train, Y_train)
    train_pred, valid_pred, pred = get_prediction(X_train, X_valid, X, model)
    disp_scores(Y_valid, valid_pred)
    return (train_pred, valid_pred, pred)

In [51]:
def get_plot_dataframe(X_train, X_valid, train_pred, valid_pred, pred):
    """学習とテスト確認用 DataFrame 作成"""
    cols = ["id", "date", "country", "store", "product"]
    # オリジナル
    df_plt = df_train[cols].copy()
    df_plt["num_sold"] = df_train["num_sold"]
    df_plt["kind"] = "orig"
    # 学習結果
    df_train_pred = pd.merge(X_train, df_train, left_index=True, right_index=True)[cols]
    df_train_pred["num_sold"] = train_pred
    df_train_pred["kind"] = "train"
    # 検証結果
    df_valid_pred = pd.merge(X_valid, df_train, left_index=True, right_index=True)[cols]
    df_valid_pred["num_sold"] = valid_pred
    df_valid_pred["kind"] = "valid"
    # 実際の予測
    df_pred = df_test.copy()
    df_pred["num_sold"] = pred
    df_pred["kind"] = "test"

    df_plt = pd.concat([df_plt, df_train_pred, df_valid_pred, df_pred]).reset_index(drop=True)
    # print(df_plt.shape)
    return df_plt

In [52]:
## LightGBM
model_name = "LightGBM"
lgbm = LGBMRegressor(data_sample_strategy='goss', max_depth=15, n_estimators=150, num_leaves=31, force_row_wise=True)    # warning 抑制
train_pred, valid_pred, pred = learning(lgbm, X_train, y_train, X_valid, y_valid, X_test)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Total Bins 627
[LightGBM] [Info] Number of data points in the train set: 82125, number of used features: 107
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 166.786581
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Test Scores
	Explained variance: 0.9772829466819039
	Mean a

In [53]:
df_plt = get_plot_dataframe(X_train, X_valid, train_pred, valid_pred, pred)
csp, df_buf = select_csp(df_plt, [0, 0, 0])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

In [54]:
## XGB
model_name = "XGB"
xgb = XGBRegressor()
train_pred, valid_pred, pred = learning(xgb, X_train, y_train, X_valid, y_valid, X_test)

Test Scores
	Explained variance: 0.9772528934365823
	Mean absolute error: 17.64675799086758
	Mean squared error: 1069.9881643835618
	R2 score: 0.9728540124582837
	SMAPE score: 12.025171820347452


In [55]:
df_plt = get_plot_dataframe(X_train, X_valid, train_pred, valid_pred, pred)
csp, df_buf = select_csp(df_plt, [0, 0, 0])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

In [78]:
## Category Boosting
model_name = "CatBoost"
catb = CatBoostRegressor()
train_pred, valid_pred, pred = learning(catb, X_train, y_train, X_valid, y_valid, X_test)

Learning rate set to 0.082161
0:	learn: 169.0945442	total: 3.55ms	remaining: 3.54s
1:	learn: 156.3678195	total: 6.89ms	remaining: 3.44s
2:	learn: 144.6896662	total: 10.3ms	remaining: 3.41s
3:	learn: 134.1189866	total: 13.7ms	remaining: 3.41s
4:	learn: 124.4355586	total: 17ms	remaining: 3.38s
5:	learn: 115.7013394	total: 20.2ms	remaining: 3.35s
6:	learn: 107.7493396	total: 23.3ms	remaining: 3.31s
7:	learn: 100.4925176	total: 26.5ms	remaining: 3.28s
8:	learn: 93.9439421	total: 29.4ms	remaining: 3.23s
9:	learn: 87.9693269	total: 32.4ms	remaining: 3.21s
10:	learn: 82.6006976	total: 35.6ms	remaining: 3.2s
11:	learn: 77.7280951	total: 38.8ms	remaining: 3.19s
12:	learn: 73.2704969	total: 41.7ms	remaining: 3.16s
13:	learn: 69.3012482	total: 44.6ms	remaining: 3.14s
14:	learn: 65.7355198	total: 47.5ms	remaining: 3.12s
15:	learn: 62.5046717	total: 51.5ms	remaining: 3.17s
16:	learn: 59.4049041	total: 54.8ms	remaining: 3.17s
17:	learn: 56.8101750	total: 57.8ms	remaining: 3.15s
18:	learn: 54.3868998

In [79]:
df_plt = get_plot_dataframe(X_train, X_valid, train_pred, valid_pred, pred)
csp, df_buf = select_csp(df_plt, [0, 0, 0])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

In [81]:
X_train.columns

Index(['year', 'month', 'day', 'dayofyear', 'dayofweek', 'quarter',
       'weekofyear', 'is_weekend', 'is_month_start', 'is_month_end',
       ...
       'holiday_敬老の日', 'holiday_文化の日', 'holiday_春分の日', 'holiday_昭和の日',
       'holiday_海の日', 'holiday_秋分の日', 'month_sin', 'month_cos', 'day_sin',
       'day_cos'],
      dtype='object', length=125)

In [94]:
## Category Boosting
model_name = "CatBoost"
cat_params = {
    'n_estimators': 733, 
    'learning_rate': 0.1252297003236841, 
    'depth': 9, 
    'l2_leaf_reg': 3.783310969240252, 
    'subsample': 0.5758950122641308, 
    'colsample_bylevel': 0.1360009677406299,
    'random_state': 42,
    'verbose': False
}
catb = CatBoostRegressor(**cat_params)
train_pred, valid_pred, pred = learning(catb, X_train, y_train, X_valid, y_valid, X_test)

Test Scores
	Explained variance: 0.9774029185069903
	Mean absolute error: 17.466849315068494
	Mean squared error: 1065.9509406392694
	R2 score: 0.9729564382879459
	SMAPE score: 10.520941584588096


In [95]:
df_plt = get_plot_dataframe(X_train, X_valid, train_pred, valid_pred, pred)
csp, df_buf = select_csp(df_plt, [0, 0, 0])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

In [58]:
## RandomForest
model_name = "RandomForest"
rf = RandomForestRegressor()
train_pred, valid_pred, pred = learning(rf, X_train, y_train, X_valid, y_valid, X_test)

Test Scores
	Explained variance: 0.9777239089047262
	Mean absolute error: 17.411689497716896
	Mean squared error: 1050.1253333333334
	R2 score: 0.9733579396812015
	SMAPE score: 10.103424917940686


In [59]:
df_plt = get_plot_dataframe(X_train, X_valid, train_pred, valid_pred, pred)
csp, df_buf = select_csp(df_plt, [0, 0, 0])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

In [36]:
## Ridge
ridge = Ridge()
train_pred, valid_pred, pred = learning(ridge, X_train, Y_train, X_valid, Y_valid, X)

Test Scores
	Explained variance: 0.753603614376493
	Mean absolute error: 0.05032981394611865
	Mean squared error: 0.005122359608950401
	R2 score: 0.7535864324248567
	SMAPE score: 0.19799265465438717


In [37]:
pred

array([[  84.9970054 ],
       [  81.38259867],
       [-111.84305776],
       ...,
       [ 294.41778338],
       [ 452.87481392],
       [ 435.45568248]])

↓ 値低い製品も悪くない気がするけど、、、他にズレているところがあるのか？  
　各々で学習させてみる？

In [96]:
df_buf = df_plt[df_plt["kind"]=="valid"].drop(columns=["kind"])
df_buf.rename(columns={"num_sold": "num_sold_valid"}, inplace=True)
df_buf = pd.merge(df_buf, df_plt[df_plt["kind"]=="orig"].drop(columns=["kind"]))

In [97]:
df_buf.head(3)

,id,date,country,store,product,num_sold_valid,num_sold
0,109575,2021-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,41,37
1,109576,2021-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,47,37
2,109577,2021-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,10,5


In [98]:
buf = []
for c in df_buf["country"].unique():
    for s in df_buf["store"].unique():
        for p in df_buf["product"].unique():
            df_p = df_buf[(df_buf["country"]==c) & (df_buf["store"]==s) & (df_buf["product"]==p)]
            rmse = mean_squared_error(df_p["num_sold"], df_p["num_sold_valid"], squared=False)
            smape = get_smape(df_p["num_sold"], df_p["num_sold_valid"])
            buf.append([c, s, p, rmse, smape])
df_mse = pd.DataFrame(buf, columns=["country", "store", "product", "rmse", "smape"])            

In [106]:
print(model_name, df_mse["smape"].mean())
df_mse.sort_values("smape")

CatBoost 10.520941584588094


,country,store,product,rmse,smape
70,Spain,Kagglazon,Using LLMs to Improve Your Coding,29.327812,4.680826
73,Spain,Kagglazon,Using LLMs to Win More Kaggle Competitions,25.343881,4.830492
71,Spain,Kagglazon,Using LLMs to Train More LLMs,29.456678,4.841315
74,Spain,Kagglazon,Using LLMs to Write Better,23.234952,4.935809
55,Japan,Kagglazon,Using LLMs to Improve Your Coding,39.074323,4.976624
...,...,...,...,...,...
32,Estonia,Kaggle Learn,Using LLMs to Win Friends and Influence People,2.236068,18.524352
35,Estonia,Kaggle Store,Using LLMs to Improve Your Coding,22.062986,18.606990
31,Estonia,Kaggle Learn,Using LLMs to Train More LLMs,13.768338,18.716147
41,Estonia,Kagglazon,Using LLMs to Train More LLMs,79.229284,19.172193


In [71]:
df_mse.sort_values("smape")

,country,store,product,rmse,smape
70,Spain,Kagglazon,Using LLMs to Improve Your Coding,30.145583,4.822031
68,Spain,Kaggle Store,Using LLMs to Win More Kaggle Competitions,7.173773,4.987388
73,Spain,Kagglazon,Using LLMs to Win More Kaggle Competitions,26.281980,5.006719
63,Spain,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,4.831121,5.019711
48,Japan,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,5.834005,5.053633
...,...,...,...,...,...
31,Estonia,Kaggle Learn,Using LLMs to Train More LLMs,13.938416,18.600673
30,Estonia,Kaggle Learn,Using LLMs to Improve Your Coding,14.612905,18.780135
34,Estonia,Kaggle Learn,Using LLMs to Write Better,11.237779,18.915875
41,Estonia,Kagglazon,Using LLMs to Train More LLMs,79.167293,18.934614


In [97]:
CSP

{'country': ['Argentina', 'Canada', 'Estonia', 'Japan', 'Spain'],
 'store': ['Kaggle Learn', 'Kaggle Store', 'Kagglazon'],
 'product': ['Using LLMs to Improve Your Coding',
  'Using LLMs to Train More LLMs',
  'Using LLMs to Win Friends and Influence People',
  'Using LLMs to Win More Kaggle Competitions',
  'Using LLMs to Write Better']}

In [107]:
# csp, df_buf = select_csp(df_plt, [0, 0, 2])
csp, df_buf = select_csp(df_plt, [4, 2, 0])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

In [108]:
csp, df_buf = select_csp(df_plt, [2, 1, 2])
ttl = f"{[csp[k][0] for k in csp.keys() if len(csp[k]) < 2]} ({model_name})"
p = make_graph.make_trend(df_buf, "num_sold", "kind", 900, 300, ttl)
show(p)

↑　絶対値が大きいと、RMSEとしては悪い方向になるかも？

submit

In [109]:
print(df_test.shape, df_subm.shape, len(pred))

(27375, 6) (27375, 2) 27375


In [110]:
# df_subm = pd.merge(df_subm.drop(columns=["num_sold"]), df_test[["id", "num_sold"]].copy(), how="left")
df_subm["num_sold"] = pred
# df_subm["num_sold"] = ((pred * 2 + 1) // 2).astype(int)    # 四捨五入int
df_subm["num_sold"].isna().sum()

0

In [111]:
df_subm

,id,num_sold
0,136950,42
1,136951,47
2,136952,10
3,136953,36
4,136954,35
...,...,...
27370,164320,710
27371,164321,695
27372,164322,109
27373,164323,649


In [112]:
df_subm.to_csv("./submission.csv", index=False)

`Public score: 50.61907` year入れて、CatBoost。でもまだ50点台。。。  
`Public score: 50.35902` RandomForest。CatBoostより下振れしてなさそうに見えたけど。。
